# News

In [175]:
!pip install google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 
!pip install --upgrade google-auth-oauthlib
!pip install openai

## Channel Definitions and Setups

In [176]:
# Dictionary of channel IDs and their corresponding names
channels = {
    "CNBC_TV": "UCrp_UI8XtuYfpiqluWLD7Lw",
    "DeepWater": 'UCQCNLsdpDV1XSHH4V8WQuPA',
    "BloombergTechnology": 'UCrM7B7SL_g1edFOnmj-SDKg',
    "YahooFinance": 'UCEAZeUIeJs0IjQiqTCdVSIg'
}

# Function to get channel name from ID
def get_channel_name_by_id(channel_id, channels):
    for name, id in channels.items():
        if id == channel_id:
            return name
    return None

# Example usage
channel_id = channels["CNBC_TV"]  # Change this to the desired channel ID
channel_name = get_channel_name_by_id(channel_id, channels)

print(f"Channel ID: {channel_id}")
print(f"Channel Name: {channel_name}")

Channel ID: UCrp_UI8XtuYfpiqluWLD7Lw
Channel Name: CNBC_TV


In [177]:
# Fetch videos
## Getting the videos from the CNBC TV channel
period_type = 'today'  # 'today', 'days', 'weeks', 'months'
number = 4  # The 'today' setting does not use 'number', adjust if using other settings
start_date, end_date = fetch_videos.get_date_range(period_type, number)
df_videos_today = fetch_videos.fetch_videos(start_date, end_date, channel_id)

In [178]:
# Assume df_videos is your initial DataFrame loaded with video data
hour_range1 = '0-8'
hour_range2 = '8-10'
hour_range3 = '10-12'
hour_range4 = '12-14'
hour_range5 = '14-16'
hour_range6 = '16-24'

# Filter videos from the last 3 days
filtered_df = fetch_videos.filter_videos_by_date_and_time(df_videos_today, 'today', 1)
fetch_videos.display_df(filtered_df)

Title,Published At,Duration (Min),Video ID
What Meta's earnings beat may signal for Apple and Amazon,2024-08-01 06:20:34-05:00,5,w3UtIMpGHEg
Evans: Use volatility to enter into the market at a lower price point,2024-08-01 06:20:38-05:00,3,EZTv23lrc_o
Shue: Mega-cap tech and the AI story can co-exist with small caps,2024-08-01 06:25:31-05:00,4,K0nbMmAowpg
How a second Trump presidency could impact the Fed and the economy,2024-08-01 06:25:33-05:00,5,zM2xEFMcErA
Energy Aspects: Middle East risks to oil should fade in coming weeks,2024-08-01 06:26:39-05:00,4,wY8gD9LQSoc
Moody's: Could be an issue if the Fed doesn't cut rates in September,2024-08-01 06:26:55-05:00,5,pGGEnjkfKRw
23andMe CEO files proposal to take company private as stock craters,2024-08-01 07:01:41-05:00,2,osKK4TzkheM
"A September rate cut is 'far more likely than not' at this point, says Roger Ferguson",2024-08-01 07:08:54-05:00,6,cyNMfi2OQNU
"Martha Stewart on the Paris Olympics, relationship with Snoop Dogg and business ventures",2024-08-01 07:09:40-05:00,7,VbALeWrnzps
Bank of England cuts interest rates for first time in over four years,2024-08-01 07:26:40-05:00,3,BTeqHyOuQp0


## Get video transcripts

In [179]:
# Add transcripts to the DataFrame
df_videos_with_transcripts = fetch_videos.add_transcripts_to_df(filtered_df)

An error occurred for video ID E53xiNAmnrs: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=E53xiNAmnrs! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!


In [180]:
#fetch_videos.display_df(df_videos_with_transcripts)

In [181]:
#fetch_videos.save_df_to_html(df_videos_with_transcripts, 'videos_with_transcripts.html')

## AI Summary 

In [182]:
# Import necessary libraries
import openai
import os
import pandas as pd

# Ensure you have set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

In [183]:
def get_summary(text, client, task):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a financial analyst."},
                {"role": "user", "content": f"Please {task} for the following text:\n\n{text}"}
            ],
            max_tokens=3000
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except openai.OpenAIError as e:  # Catch the general OpenAIError
        if "maximum context length" in str(e):
            print(f"Warning: Context length exceeded for transcript. Returning an empty summary.")
            return "Context length exceeded. Summary not available."
        else:
            raise e  # Re-raise other types of OpenAI errors

def get_summary_all_transcripts(df, client, task):
    """Generates summaries or "No summary" messages for transcripts."""
    for index, row in df.iterrows():
        transcript = row['Transcript']
        summary = get_summary(transcript, client, task)

        # Check if the summary is empty or the placeholder string
        if not summary or summary == "Context length exceeded. Summary not available.":
            summary = "No summary"

        df.at[index, 'Summary'] = summary
    return df

# Example task
task = """I would like you to summarize the transcript with the following instructions: 
First categorize the video content.The category should be one of the following: Crypto, Macro, Politics, Technology Stocks, or Other. 
Then summarize the stocks that are mentioned in this video.
Then provide key takeaways in a bullet point format.
Finally, use one sentence to summarize the overall sentiment of the stocks mentioned in the video.
Please print the summary in a human-readable format like the following: 
Category: Technology Stocks
Stock mentioned: Nvidia, Tesla, Apple
Key takeaways:
* Nvidia reported record revenue growth.
* Tesla announced a new electric vehicle model.
* Apple released a new software update which will make its products more secure.
Sentiment: Cautiously Positive
"""

# Get summaries for all transcripts
df_summaries = get_summary_all_transcripts(df_videos_with_transcripts, client, task)

In [184]:
summary_file_name = f'summaries_{channel_name}.csv'
df_summaries.to_csv(summary_file_name, index=False)

In [185]:
# Display the summaries without the original transcripts
df_summaries_display = df_summaries[['Title', 'Summary']]
df_summaries_display

,Title,Summary
33,What Meta's earnings beat may signal for Apple...,Category: Technology Stocks\nStocks mentioned:...
68,Evans: Use volatility to enter into the market...,Category: Macro\n\nStock mentioned: Microsoft\...
3,Shue: Mega-cap tech and the AI story can co-ex...,Category: Macro\nStock mentioned: None\nKey ta...
14,How a second Trump presidency could impact the...,Category: Macro\nStock mentioned: N/A\nKey tak...
6,Energy Aspects: Middle East risks to oil shoul...,Category: Macro\n\nStocks mentioned: N/A\n\nKe...
...,...,...
49,Still questions around how Amazon will take ad...,"Category: Other\n\nStocks mentioned: Apple, Am..."
27,Health care bucks sell-off as Eli Lilly soars:...,Category: Other\n\nStocks mentioned: Eli Lilly...
53,"10-year Treasury speaks louder than words, say...",Category: Macro\n\nStock mentioned: Not applic...
67,"We're seeing a lot of growth in data centers, ...",Category: Other\nStock mentioned: None\nKey ta...


## Send emails

In [186]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pandas as pd
import os
import fetch_videos

email_user = os.getenv('EMAIL_USER')
email_password = os.getenv('EMAIL_PASSWORD')
email_send1 = 'sliu810@gmail.com'
email_send2 = 'zhengwang827@gmail.com'

recipients = [email_send1]
# Set up the MIME
message = MIMEMultipart()
message['From'] = email_user
message['To'] = email_send1
message['Subject'] = f'summaries_{channel_name}_{fetch_videos.get_formated_date_today()}'

# load df_summaries from disk and get the html content
df_summaries = pd.read_csv(summary_file_name)
html_content= fetch_videos.get_html_content_summary_only(df_summaries)

# Attach the HTML content to the email
message.attach(MIMEText(html_content, 'html'))

# Function to send email
def send_email():
    try:
        server = smtplib.SMTP('smtp-mail.outlook.com', 587)  # Outlook SMTP server
        #server.set_debuglevel(1)  # Enable debugging output
        server.starttls()
        server.login(email_user, email_password)
        text = message.as_string()
        server.sendmail(email_user, recipients, text)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Send the email
send_email()

Email sent successfully!


In [187]:
import fetch_videos

df_summaries = pd.read_csv(summary_file_name)
html_content = fetch_videos.get_html_content_summary_only(df_summaries)
print(html_content)

summary_file_name_html = f'summaries_{channel_name}_{fetch_videos.get_formated_date_today()}.html'

# Save the HTML content to an HTML file
with open(summary_file_name_html, 'w') as file:
    file.write(html_content)

<div><h3><a href="https://www.youtube.com/watch?v=w3UtIMpGHEg" target="_blank">What Meta's earnings beat may signal for Apple and Amazon</a></h3><p><br><b>Category:</b> Technology Stocks</li><br><b>Key Stocks mentioned:</b><ul> Meta (formerly Facebook), Alphabet (Google), Apple, Amazon</li><br><b>Key takeaways:</b><ul></li>* Meta platforms reported a solid quarter with revenue and operating profit beats.</li>* Differences noted in AI strategies between Meta Platforms' Mark Zuckerberg and Alphabet's Sundar Pichai.</li>* Concern that Apple may be over-hyping AI capabilities in upcoming iPhone release.</li>* Amazon expected to show increased margins in AWS and retail business profitability.</li>Sentiment: Mixed sentiment, with positive outlook for Meta Platforms and Amazon, but caution regarding Apple's future performance.</li></ul></p></div>
<div><h3><a href="https://www.youtube.com/watch?v=EZTv23lrc_o" target="_blank">Evans: Use volatility to enter into the market at a lower price point

## Random one off experiement 

In [188]:
# print(transcript)

In [189]:
# transcript = fetch_videos.get_transcript('GabBf791bdY')
# print(transcript)
# # Example task
# task2 = """I would like you to summarize the text which describes Tesla FSD.
# Please say in what senario FSD performed well and waht senario it didn't perform well.
# Format like this:
# Title: 
# Key takeaways:
# * FSD performed well in ...
# * FSD had some challenges in ....
# Improvement over pervious version if any.
# """
# task3 = """could you print the transcript with two person speaking into a human readible format and maintain the original content?"""

# summary = get_summary(transcript, client, task3)
# print(summary)

In [190]:
def get_summary_in_chunks(transcript, client, task, chunk_size=500):
    """
    Generates summaries in chunks to avoid truncation.

    Parameters:
    - transcript (str): The full transcript text.
    - client (object): The client to use for generating summaries.
    - task (str): The task identifier for the summary generation.
    - chunk_size (int): The maximum size of each chunk.

    Returns:
    - str: Combined summary of all chunks.
    """
    import textwrap

    # Split the transcript into chunks
    chunks = textwrap.wrap(transcript, chunk_size)
    full_summary = ""

    for chunk in chunks:
        summary = get_summary(chunk, client, task)
        full_summary += summary + "\n"

    return full_summary

In [191]:
transcript = fetch_videos.get_transcript('GUM2FSuqOoE')
# print(transcript)
# Example task
task2 = """I would like you to summarize the text which describes Tesla FSD.
Please say in what senario FSD performed well and waht senario it didn't perform well.
Format like this:
Title: 
Key takeaways:
* FSD performed well in ...
* FSD had some challenges in ....
Improvement over pervious version if any.
"""
task3 = """print the transcript in a more human reaible fashion"""

summary = get_summary(transcript, client, task3)
print(summary)

Context length exceeded. Summary not available.
